In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install selenium

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.qjcZHDyopV/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.38HpYNhbC4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.4EeASlE7uV/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    prefs = {'download.default_directory' : download_directory}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(options=options)
    return driver

## Free MIDI

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import time
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup

links = []
base_url = "https://freemidi.org"
initial_url = "https://freemidi.org/genre-dance-eletric"
download_directory = "/content/drive/MyDrive/palitech_5/2 kurs/диплом/data/freemidi"


if not os.path.exists(download_directory):
    os.makedirs(download_directory)

response = requests.get(initial_url)
soup = BeautifulSoup(response.text, 'html.parser')

genre_band_containers = soup.find_all('div', class_='genre-band-container')
print(genre_band_containers[0])
for genre_band_container in genre_band_containers:
    artist_url = genre_band_container.find('div', class_='genre-link-text')
    artist_song_url = urljoin(base_url, artist_url.a['href'])
    artist_song_page = requests.get(artist_song_url)
    artist_song_soup = BeautifulSoup(artist_song_page.text, 'html.parser')

    artist_song_cells = artist_song_soup.find_all('div', class_='artist-song-cell')
    for artist_song_cell in artist_song_cells:
        midi_file_url = urljoin(base_url, artist_song_cell.span.a['href'])

        links.append(urljoin(base_url, midi_file_url))

print("Links adding process completed.")

<div class="genre-band-container"><a name="2" style="padding-bottom:0;padding-top:0">
<div class="genre-link-text"><a href="artist-107-2-brothers-on-the-4th-floor">2 Brothers On The 4th Floor</a></div>
</a></div>
Links adding process completed.


In [ ]:
from selenium.webdriver.common.by import By
driver = web_driver()

for link in links:
  driver.get(link)
  el = driver.find_element(By.ID, "downloadmidi")
  el.click()
driver.quit()

## MIDIS101


In [29]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import time
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup

links_101 = []
base_url = "https://www.midis101.com"
initial_url = "https://www.midis101.com/search/techno/pg-"
download_directory = "/content/drive/MyDrive/palitech_5/2 kurs/диплом/data/midis101"


if not os.path.exists(download_directory):
    os.makedirs(download_directory)



for i in range(1, 14):
  response = requests.get(initial_url + str(i))
  soup = BeautifulSoup(response.text, 'html.parser')
  song_containers = soup.find_all('tr')[1:]
  for song_container in song_containers:
      song = song_container.find('td')
      artist_song_url = urljoin(base_url, song.a['href'])
      artist_song_page = requests.get(artist_song_url)
      artist_song_soup = BeautifulSoup(artist_song_page.text, 'html.parser')

      midi_file_url = urljoin(base_url, artist_song_soup.find('a', class_="btn btn-primary btn-lg btn-block p-4")['href'])
      links_101.append(urljoin(base_url, midi_file_url))

  print("Links adding process completed.")

Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.
Links adding process completed.


In [37]:
from selenium.webdriver.common.by import By
driver = web_driver()

for link in links_101:
  print(link)
  driver.get(link)
driver.quit()

https://www.midis101.com/download/93920-medley-titanic-techno
https://www.midis101.com/download/86445-russian-yam-demos-techno
https://www.midis101.com/download/86202-russian-s-penkin-besamemuchointechno
https://www.midis101.com/download/83024-mls-pr-technotronic-pumpupthejam1
https://www.midis101.com/download/58829-technos-radadiradada
https://www.midis101.com/download/58828-techno-triphop
https://www.midis101.com/download/58827-techno-trance
https://www.midis101.com/download/58826-techno-thehouse
https://www.midis101.com/download/58825-techno-texmex2
https://www.midis101.com/download/58824-techno-texmex1
https://www.midis101.com/download/58823-techno-texmex
https://www.midis101.com/download/58822-techno-techpop
https://www.midis101.com/download/58821-techno-techno2
https://www.midis101.com/download/58820-techno-techno1a
https://www.midis101.com/download/58819-techno-techno1
https://www.midis101.com/download/58818-techno-techno
https://www.midis101.com/download/58817-techno-techno-2
h